In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow
from tensorflow import keras
# from tensorflowkeras.utils.vis_utils import plot_model
import random
from sklearn.model_selection import train_test_split
from scipy import ndimage
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization,Conv2D,MaxPooling2D,Activation,Dropout,Lambda,Dense,Flatten,Input

import tensorflow as tf

import time
import psutil
import csv
import gc
import pickle

2024-05-01 12:10:29.592243: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-01 12:10:29.594907: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-01 12:10:29.624046: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-01 12:10:30.080997: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
class custom_MTL():
    def __init__(self) -> None:
        
        self.layer_list=[]
        self.weight_set=False
        self.partition_done = False
        x_shape, y_angle_shape, y_cifar10_shape=(50000, 32, 32, 3), (50000, 4), (50000, 10)
        angle_classifier_no=4
        cifar10_classifier_no=10
        inputs = keras.Input((32, 32, 3))
        conv_base = self.get_convbase(inputs)
        angle_classifier = self.get_classifier(conv_base, angle_classifier_no, "angle",count=15)
        cifar10_classifier = self.get_classifier(conv_base, cifar10_classifier_no, "cifar10",count=22)
        self.model = Model(
            inputs=inputs, 
            outputs=[cifar10_classifier, angle_classifier]
        )
        tf.keras.utils.plot_model(self.model, to_file='final_mtl_model.png', show_shapes=True,show_layer_names=True)
    
   
    def get_convbase(self,inputs):
    
    # reg = keras.regularizers.l2(1e-4)
    
    # initializer = keras.initializers.HeNormal()


        x = Conv2D(16, (3, 3), padding="same",name='1_conv2D_1')(inputs)
        x = Activation("relu",name='2_activation_1')(x)
        x = BatchNormalization(axis=-1,name='3_batch_norm_1')(x)
        x = MaxPooling2D(pool_size=(3, 3),name='4_maxPool_1')(x)
        x = Dropout(0.25,name='5_dropout_1')(x)
        
        x = Conv2D(32, (3, 3), padding="same",name='6_conv2D_2')(x)
        x = Activation("relu",name='7_activation_2')(x)
        x = BatchNormalization(axis=-1,name='8_batch_norm_2')(x)
        x = MaxPooling2D(pool_size=(2, 2),name='9_maxPool_2')(x)
        x = Dropout(0.25,name='10_dropout_2')(x)
        
        x = Conv2D(32, (3, 3), padding="same",name='11_conv2D_3')(x)
        x = Activation("relu",name='12_activation_3')(x)
        x = BatchNormalization(axis=-1,name='13_batch_norm_3')(x)
        x = MaxPooling2D(pool_size=(2, 2),name='14_maxPool_3')(x)
        x = Dropout(0.25,name='15_dropout_3')(x)
        
        return x
    
    def get_classifier(self,x, class_no, name,count):
    
        x = Flatten(name=f'{count+1}_layer')(x)
        x = Dense(128,name=f'{count+2}_layer')(x)
        x = Activation("relu",name=f'{count+3}_layer')(x)
        x = BatchNormalization(name=f'{count+4}_layer')(x)
        x = Dropout(0.5,name=f'{count+5}_layer')(x)
        x = Dense(class_no,name=f'{count+6}_layer')(x)
        x = Activation("softmax", name=name)(x)

        return x

    def load_weights(self):
        self.model.load_weights('./custom_mtl_model.h5')
        self.weight_set=True
        return
    def load_input(self):
        self.input_data = np.random.rand(1, 32, 32, 3).astype(np.float32)
        self.input_loaded=True
        return self.input_data
        
    def save_pickeled_layers(self):
        if not self.weight_set:
            self.loadWeights()

        
        if not self.partition_done:
            self.make_partition()
        save_dir='./../pickle_layers'
        for i in range(len(self.layer_list)):
            fname=f'./{save_dir}/custum_mtl_layer_{i}.pkl'
            layer_weights_and_config = {
                'weights': self.layer_list[i].get_weights(),
                'config': tf.keras.layers.serialize(self.layer_list[i])}
            with open(fname, 'wb') as f:
                pickle.dump(layer_weights_and_config, f)
                
                
    def make_partition(self):
        self.layer_list=[]
        self.NO_OF_LAYERS= len(self.model.layers)
        
        for i in range(self.NO_OF_LAYERS):
            self.temp_layer=self.model.layers[i]
            self.layer_list.append(self.temp_layer)
            
        self.partition_done = True
        print('\_______Partitioning Done')
        
    def execute_predict(self,input_data):
        st1=time.perf_counter()
        out=self.model.predict(input_data)
        et1=time.perf_counter()
        el1=et1-st1
        print(f'Elapsed Time: {el1}')
        return out
    
    def print_lays(self):
        for lay in self.model.layers:
            self.layer_list.append(lay)
            print(lay)
        print(f'Total number of Layers : {len(self.layer_list)}')
    def execute_lbl(self,input_data):
        st2=time.perf_counter()
        out=buffer=input_data
        for idx in range(1,len(self.model.layers)):
            
            if idx <= 15:
                print(f'Executing index {idx} : {self.model.layers[idx]}')
                out=buffer=self.model.layers[idx](out)
            elif idx in [16,18,20,22,24,26,28]:
                print(f'Executing index {idx} : {self.model.layers[idx]}')
                out=self.model.layers[idx](out)
            elif idx in [17,19,21,23,25,27,29]:
                print(f'Executing index {idx} : {self.model.layers[idx]}')
                buffer=self.model.layers[idx](buffer)
        et2=time.perf_counter()
        el2=et2-st2
        print(f'Elapsed Time: {el2}')
        return out,buffer
    
    def gte_input_list(self,input_data):
        self.input_list=[]
        st2=time.perf_counter()
        out=buffer=input_data
        self.input_list.append(input_data)
        for idx in range(1,len(self.model.layers)):
            # print(f'Executing index {idx} :  {self.model.layers[idx]}')
            if idx <= 15:
                self.input_list.append(out)
                out=buffer=self.model.layers[idx](out)
            elif idx in [16,18,20,22,24,26,28]:
                self.input_list.append(out)
                out=self.model.layers[idx](out)
            elif idx in [17,19,21,23,25,27,29]:
                self.input_list.append(buffer)
                buffer=self.model.layers[idx](buffer)
        et2=time.perf_counter()
        el2=et2-st2
        print(f'Elapsed Time: {el2}')
        return self.input_list
    
    def execute_on_core(self,layer_id,input_data):
        # dummy_data=dummy_data
        # print(self.layer_list[layer_id].name)
        self.temp_out=self.layer_list[layer_id](input_data)
        
        return self.temp_out
        
    


In [6]:
NO_OF_LAYERS=30
NO_OF_CPU=24

In [7]:
def save_2d_list_to_csv(data, file_name):
    """
    Save a 2D list to a CSV file.

    Parameters:
        data (list): The 2D list to be saved.
        file_name (str): The name of the CSV file to be created.
    """
    with open(file_name, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerows(data)

    print(f'CSV file "{file_name}" has been created.')

In [8]:
def calculate_mean(num_list):

    if not num_list:
        return None  # Return None if the list is empty

    total = sum(num_list)  # Calculate the sum of all numbers in the list
    mean = total / len(num_list)  # Calculate the mean by dividing the sum by the number of elements
    return mean


In [9]:
image = np.random.rand(1,32,32,3)

In [10]:
obj=custom_MTL()
obj.load_weights()
# out=obj.model.predict(image)
# obj.print_lays()
obj.execute_predict(image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Elapsed Time: 0.09914163395296782


[array([[0.08216629, 0.07917303, 0.0721583 , 0.10970639, 0.10685154,
         0.10086906, 0.16356683, 0.08192592, 0.09662664, 0.10695593]],
       dtype=float32),
 array([[0.23763563, 0.22090478, 0.26563695, 0.27582258]], dtype=float32)]

In [11]:
obj.make_partition()
obj.save_pickeled_layers()

\_______Partitioning Done


In [21]:
INPUT_LIST=obj.gte_input_list(image)
len(INPUT_LIST)

Elapsed Time: 0.008073513978160918


30

In [22]:
EXP=25
dummy=[0]*NO_OF_CPU
layers_ex=[dummy]
layers_ex_ave=[dummy]
for l in range(1,NO_OF_LAYERS):
    cpu_ex=[]
    cpu_ex_avg=[]
    for cpu in range(NO_OF_CPU):
        temp=[]
        for i in range(EXP):
            temp.append(compute_execution_time(obj,'execute_on_core',cpu,l,INPUT_LIST[l]))
        
        temp1=calculate_mean(temp)
        temp2=min(temp)
        cpu_ex.append(temp2)
        cpu_ex_avg.append(temp1)
    time.sleep(0.5)
    layers_ex.append(cpu_ex)
    layers_ex_ave.append(cpu_ex_avg)


save_2d_list_to_csv(layers_ex, "./readings/single_layer_profile_custom_mtl_for_percentage.csv")
save_2d_list_to_csv(layers_ex_ave, "./readings/single_layer_ave_profile_custom_mtl_for_percentage.csv")


CSV file "./readings/single_layer_profile_custom_mtl_for_percentage.csv" has been created.
CSV file "./readings/single_layer_ave_profile_custom_mtl_for_percentage.csv" has been created.
